In [1]:
import pandas as pd
import os


In [2]:
lista_cpfs = os.listdir("2020_income")

lista_cpfs

['CENTRO_OESTE2020ID.dta.pkl',
 'MG_ES_RJ2020ID.dta.pkl',
 'NORDESTE2020ID.dta.pkl',
 'NORTE2020ID.dta.pkl',
 'SP2020ID.dta.pkl',
 'SUL2020ID.dta.pkl']

In [3]:
cpf_total = pd.DataFrame()
for cpf_file in lista_cpfs:
    df_cpf = pd.read_pickle("2020_income\\"+cpf_file)
    cpf_total = pd.concat([cpf_total,df_cpf])

In [4]:
cpf_total["nu_cpf"]=cpf_total["CPF"].astype(float) 
cpf_total

,CPF,remmedr,nu_cpf
0,07513484813,1113.46,7.513485e+09
1,40802280153,1100.00,4.080228e+10
2,86234145191,1121.00,8.623415e+10
3,40646653172,2092.57,4.064665e+10
4,61503207153,1722.74,6.150321e+10
...,...,...,...
12161305,82066523020,3534.87,8.206652e+10
12161306,01884059090,2959.74,1.884059e+09
12161307,02197961063,1565.32,2.197961e+09
12161308,93102534015,1982.12,9.310253e+10


In [5]:
csup = pd.read_stata("D:\\datasets\\coorte13_ces_enem.dta")

In [6]:
#len(pd.unique(csup.no_curso))
csup.filter(like="nota")

,nota_cn,nota_ch,nota_lc,nota_mt,notas,nota_redacao
0,632.000000,659.099976,641.299988,694.299988,656.674988,700.0
1,556.099976,610.900024,467.399994,501.799988,534.049988,340.0
2,614.400024,554.200012,581.500000,799.099976,637.299988,580.0
3,NaN,NaN,NaN,NaN,NaN,NaN
4,625.400024,714.900024,577.400024,733.200012,662.725037,780.0
...,...,...,...,...,...,...
2929263,469.299988,518.900024,534.700012,443.500000,491.600006,500.0
2929264,NaN,NaN,NaN,NaN,NaN,NaN
2929265,NaN,NaN,NaN,NaN,NaN,NaN
2929266,567.599976,586.599976,575.299988,677.700012,601.799988,440.0


In [7]:
csup.drop_duplicates(subset=["nu_cpf"]).columns

Index(['co_aluno', 'nu_cpf', 'sexo', 'cor_raca', 'cor_raca_ces',
       'cor_raca_enem', 'conclusao_em', 'ano_conclusao_em', 'idade',
       'ano_nascimento', 'in_enem', 'nota_cn', 'nota_ch', 'nota_lc', 'nota_mt',
       'notas', 'nota_redacao', 'edupar', 'escpai', 'escmae', 'rdpc',
       'log_rdpc', 'renda', 'qt_hab', 'imovel', 'trabalha', 'escola_ef',
       'escola_em', 'co_curso', 'vinculo_2013', 'vinculo_2014', 'vinculo_2015',
       'vinculo_2016', 'vinculo_2017', 'vinculo_2018', 'vinculo_2019',
       'co_ies', 'categoria_adm', 'organizacao_acad', 'grau', 'modalidade',
       'co_cine_rotulo', 'no_cine_rotulo', 'co_cine_area_detalhada',
       'no_cine_area_detalhada', 'co_cine_area_especifica',
       'no_cine_area_especifica', 'co_cine_area_geral', 'no_cine_area_geral',
       'reserva_vagas_2013', 'turno_2013', 'turno_2014', 'turno_2015',
       'turno_2016', 'turno_2017', 'turno_2018', 'turno_2019',
       'financiamento_2013', 'financiamento_2014', 'financiamento_2015',
  

In [8]:
merged_csup = csup[["nu_cpf",
                    "co_curso",
                    "notas",
                    "sexo",
                    "idade",
                    "categoria_adm",
                    "co_ies",
                    "modalidade"]].merge(cpf_total[["remmedr",
                                                  "nu_cpf"]],on="nu_cpf",how="inner")

In [9]:
merged_csup = merged_csup.\
    merge(pd.read_stata("D:\\datasets\\curso2013.dta")[["co_curso",
                                                        "co_municipio_curso"]],
                                                        how="left")

merged_csup = merged_csup.merge(pd.read_csv("D:\\datasets\\brazilianGeographicDivisions.csv").\
                                                              rename(columns={"co_muni":"co_municipio_curso"})[["co_municipio_curso",
                                                                                                                "co_meso"]].drop_duplicates(),how='left')

In [12]:
import pandas as pd
import pickle
from sdv.metadata import SingleTableMetadata
from sdv.single_table import GaussianCopulaSynthesizer
from copy import deepcopy

df = deepcopy(merged_csup)  
df["co_curso"] = df["co_curso"].astype(str)+"_curso"
df["co_ies"] = df["co_ies"].astype(str)+"_ies"
df["nu_cpf"] = df["nu_cpf"].astype(str)+"_cpf"
df["co_meso"] = df["co_meso"].map('{:.0f}'.format).astype(str)+"_meso"


# Convert categorical columns to object type and fill missing values 
categorical_cols = ['sexo', 'categoria_adm', 'modalidade','co_curso','co_ies','nu_cpf','co_meso']
for col in categorical_cols:
    
    df[col] = df[col].astype('object')
    
    df[col] = df[col].fillna('missing')


# Gaussian Copula synthesizer
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df)

# Gaussian Copula Synthesizer with metadata.
synth = GaussianCopulaSynthesizer(metadata)

print("Fitting the Gaussian Copula model on your real data...")
synth.fit(df)



with open('synth_model.pkl', 'wb') as f:
    pickle.dump(synth, f)
print("Fitted synthesizer model saved to 'synth_model.pkl'.")


with open('synth_model.pkl', 'rb') as f:
    loaded_synth = pickle.load(f)
synthetic_df = loaded_synth.sample(num_rows=len(df))
synthetic_df.to_csv("synthetic_data.csv", index=False)



#synthetic_df.to_csv("synthetic_data.csv", index=False)
#print("Synthetic data generated and saved to 'synthetic_data.csv'.")
#print(synthetic_df.head())


c:\Users\erugg\anaconda3\Lib\site-packages\sdv\single_table\base.py:119: FutureWarning:

The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.

c:\Users\erugg\anaconda3\Lib\site-packages\sdv\single_table\base.py:104: UserWarning:

We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.



Fitting the Gaussian Copula model on your real data...
Fitted synthesizer model saved to 'synth_model.pkl'.


In [ ]:
metadata

{'nu_cpf': {'sdtype': 'unknown', 'pii': True},
 'co_curso': {'sdtype': 'categorical'},
 'notas': {'sdtype': 'numerical'},
 'sexo': {'sdtype': 'categorical'},
 'idade': {'sdtype': 'numerical'},
 'categoria_adm': {'sdtype': 'categorical'},
 'co_ies': {'sdtype': 'categorical'},
 'modalidade': {'sdtype': 'categorical'},
 'remmedr': {'sdtype': 'numerical'},
 'co_municipio_curso': {'sdtype': 'numerical'},
 'co_meso': {'sdtype': 'categorical'}}

In [ ]:
import pandas as pd
import numpy as np
from pyfixest.estimation import feols


# 1. Load your data
df = synthetic_df.head(500000)

# 2. Clean or drop rows that have missing values for variables used in the regression
#    (If 'remmedr' or 'idade' is zero or negative, log() is invalid, so remove those too.)
df = df.dropna(subset=['remmedr', 'idade', 'notas', 'sexo', 'co_curso'])
df = df[(df['remmedr'] > 0) & (df['idade'] > 0)]

# 3. Create the new columns needed for the regression
df['log_income'] = np.log(df['remmedr'])       # log(remmedr)
df['lage'] = np.log(df['idade'])               # log(idade)
df['female'] = (df['sexo'] == 'Feminino').astype(int)

# 4. Fit OLS with co_curso as a fixed effect via categorical dummies (C(co_curso))
#model = smf.ols("log_income ~ lage + notas + female + C(co_curso)", data=df).fit()
fit = feols("log_income ~ lage + notas + female | co_curso", data=df)
fit.summary()
# 5. Print summary
#print(model.summary())


###

Estimation:  OLS
Dep. var.: log_income, Fixed effects: co_curso
Inference:  CRV1
Observations:  274925

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:--------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| lage          |      0.388 |        0.021 |    18.530 |      0.000 |  0.347 |   0.429 |
| notas         |      0.004 |        0.000 |    41.758 |      0.000 |  0.004 |   0.005 |
| female        |     -0.156 |        0.012 |   -12.616 |      0.000 | -0.180 |  -0.132 |
---
RMSE: 0.789 R2: 0.769 R2 Within: 0.034 


In [ ]:
merged_csup.head(500000)#.co_curso.value_counts()#.sum()#.co_curso.value_counts()

,nu_cpf,co_curso,notas,sexo,idade,categoria_adm,co_ies,modalidade,remmedr
0,9.161980e+09,57872,656.674988,Feminino,22.0,Publica federal,587,Presencial,1465.15
1,9.161980e+09,57872,656.674988,Feminino,22.0,Publica federal,587,Presencial,1134.70
2,1.696350e+10,115440,534.049988,Masculino,36.0,Publica federal,571,Presencial,1375.40
3,2.748279e+09,15852,662.725037,Feminino,21.0,Publica federal,694,Presencial,2315.97
4,2.748279e+09,15852,662.725037,Feminino,21.0,Publica federal,694,Presencial,2048.87
...,...,...,...,...,...,...,...,...,...
499995,6.877620e+10,111720,NaN,Masculino,32.0,Privada com fins lucrativos,1472,EaD,5742.73
499996,3.601899e+09,102375,NaN,Feminino,41.0,Privada com fins lucrativos,163,Presencial,1374.95
499997,3.559241e+10,19522,NaN,Masculino,26.0,Privada com fins lucrativos,322,Presencial,2842.99
499998,6.545598e+09,88484,507.225006,Feminino,32.0,Privada com fins lucrativos,2271,Presencial,2905.47


In [ ]:
synthetic_df#.head(500000).co_curso.value_counts()

,nu_cpf,co_curso,notas,sexo,idade,categoria_adm,co_ies,modalidade,remmedr
0,1202868_curso_cpf,1204112_curso,NaN,Feminino,32.0,Privada com fins lucrativos,9263_curso_ies,Presencial,3864.67
1,346623_curso_cpf,1264882_curso,NaN,Masculino,17.0,Publica federal,1149798_curso_ies,Presencial,9756.20
2,91031_curso_cpf,97990_curso,NaN,Feminino,17.0,Publica federal,97990_curso_ies,Presencial,896.44
3,1128647_curso_cpf,16_curso,NaN,Masculino,32.0,Publica federal,20938_curso_ies,Presencial,115.85
4,12586_curso_cpf,118096_curso,NaN,Feminino,31.0,Publica federal,114042_curso_ies,Presencial,182.70
...,...,...,...,...,...,...,...,...,...
499995,41578_curso_cpf,101747_curso,648.756470,Masculino,32.0,Privada com fins lucrativos,73410_curso_ies,Presencial,934.86
499996,71073_curso_cpf,3150_curso,658.244141,Masculino,26.0,Publica estadual,2214_curso_ies,Presencial,714.42
499997,1150343_curso_cpf,38579_curso,NaN,Masculino,32.0,Privada com fins lucrativos,69754_curso_ies,Presencial,2424.42
499998,80865_curso_cpf,1164570_curso,NaN,Feminino,18.0,Publica estadual,1164570_curso_ies,EaD,2180.54


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Load your dataset
file_path = "your_data.csv"  # Replace with your actual file
df = pd.read_csv(file_path)



In [ ]:
#merged_csup = csup[["nu_cpf","co_curso"]].merge(cpf_total[["radiccnpj","dtadmissao","diadesli","mesdesli","nu_cpf"]].drop_duplicates(subset=["nu_cpf"]),on="nu_cpf",how="inner")
merged_csup = csup[["nu_cpf","co_curso"]].merge(cpf_total[["radiccnpj","dtadmissao","diadesli","mesdesli","nu_cpf"]],on="nu_cpf",how="inner")

In [ ]:
merged_csup.head().columns#[["nu_cpf","CPF"]].dropna(subset='CPF')

Index(['nu_cpf', 'co_curso', 'radiccnpj', 'dtadmissao', 'diadesli',
       'mesdesli'],
      dtype='object')

## Create unique identifier

In [ ]:
import hashlib
import pandas as pd


# Convert all to string and create a unique identifier
merged_csup['unique_id'] = merged_csup.apply(lambda x: hashlib.sha256(f"{str(x['co_curso'])}_{str(x['radiccnpj'])}".encode()).hexdigest(), axis=1)


In [ ]:
import pandas as pd


# Convert 'dtadmissao' to datetime format
#merged_csup['dtadmissao'] = pd.to_datetime(merged_csup['dtadmissao'], format='%d%m%Y')

merged_csup['diadesli'] = pd.to_numeric(merged_csup['diadesli'], errors='coerce')
merged_csup['mesdesli'] = pd.to_numeric(merged_csup['mesdesli'], errors='coerce')

# Function to construct the separation date safely
def get_desligamento(row):
    if pd.notna(row['diadesli']) and pd.notna(row['mesdesli']):
        try:
            return pd.Timestamp(year=2019, month=int(row['mesdesli']), day=int(row['diadesli']))
        except ValueError:
            return pd.NaT  # Handles invalid day/month combinations
    else:
        return pd.Timestamp("2020-01-01")  # Default for NaN values

# Apply the function row-wise
merged_csup['desligamento'] = merged_csup.apply(get_desligamento, axis=1)

merged_csup.head()

,nu_cpf,co_curso,radiccnpj,dtadmissao,diadesli,mesdesli,unique_id,desligamento
0,2.748279e+09,15852,26192813,2019-02-11,11.0,5.0,4424e369d38ca58544e9b1d1627d27fe5bc892cdb29cf2...,2019-05-11
1,1.184102e+10,33738,06137579,2017-06-01,23.0,8.0,95b93caee0ff07ea6a3e07f069e3716bb0cf0fecb683f1...,2019-08-23
2,3.999433e+10,49464,59527788,2018-09-10,NaN,NaN,93f4c56267ba4953b02b1c84acfe75e38e3f3c51be3f10...,2020-01-01
3,1.404696e+10,12808,22896431,2019-11-04,NaN,NaN,0cd71dfa5086be33ebc1d00f04f8b57b49237c08b9b359...,2020-01-01
4,8.032529e+09,122906,00394502,1998-06-15,NaN,NaN,4617ae76afbc5e7d3949b95fe6ce24c759bcff23f55b0a...,2020-01-01


In [ ]:
len(pd.unique(merged_csup.nu_cpf))/len(pd.unique(csup.nu_cpf))

0.5958920172501401

In [ ]:
def get_colleagues(row, df):
    same_hash_group = df[df["unique_id"] == row["unique_id"]]  # Get same unique_id group
    overlapping = same_hash_group[
        (same_hash_group["dtadmissao"] <= row["desligamento"]) & 
        (same_hash_group["desligamento"] >= row["dtadmissao"]) &  # Overlap condition
        (same_hash_group["nu_cpf"] != row["nu_cpf"])  # Exclude itself
    ]
    return list(overlapping["nu_cpf"])

# Apply function for each row
#merged_csup["colleagues"] = merged_csup.apply(lambda row: get_colleagues(row, merged_csup), axis=1)

In [ ]:
from tqdm import tqdm
import pandas as pd

# Ensure tqdm works inside pandas apply
tqdm.pandas()

# Apply function with progress tracking
#merged_csup["colleagues"] = merged_csup.progress_apply(lambda row: get_colleagues(row, merged_csup), axis=1)
teste = merged_csup.progress_apply(lambda row: get_colleagues(row, merged_csup), axis=1)


  0%|          | 786/1740520 [00:57<35:34:59, 13.58it/s]


KeyboardInterrupt: 

In [ ]:
merged_csup.unique_id.value_counts()

unique_id
91d3642312d184e682f1c42215790eead6bc6a3261d20fd77e1e8067766b5a63    760
33e9ec4b9da2563577db0499d880171f324bdf43f9e19d3a42f3fbcb78719985    656
27b0f255c160eead9247a6e76be915bf752069a03c3935387f033113b3e0e1c7    532
8e4e28be189ea764038aa098d42c60618ec97f324de730d781440d2a39ef4cbf    434
844b62e34f6ed427be9618c93a1794e213a87872d98c69060de574636a9735a0    355
                                                                   ... 
cd95984ed58ad2049ca73b79b00e367f0842f7baf762491196a7477e0bea0435      1
aac477b29e8a9af058658592d74b864156bf69790e5189a345928082903139e5      1
c1f2b5cb198d36d55e0a5ae6e715645fa61abc92cdfb16ac7f7d3f8179f3abf9      1
df81e4d1bcbafb4c3e73b4f2ab062aacecbbe4501cbd4e736f13ea033d508273      1
9b27412dedeea54831215da61d1bf93252b5c75282520a0cfee19b8e925d8671      1
Name: count, Length: 1370530, dtype: int64

In [ ]:
merged_csup[merged_csup.unique_id=="33e9ec4b9da2563577db0499d880171f324bdf43f9e19d3a42f3fbcb78719985"]

,nu_cpf,co_curso,radiccnpj,dtadmissao,diadesli,mesdesli,unique_id,desligamento
1233029,2.251767e+10,96655,46379400,2015-05-11,NaN,NaN,33e9ec4b9da2563577db0499d880171f324bdf43f9e19d...,2020-01-01
1233231,1.571883e+10,96655,46379400,2018-02-02,NaN,NaN,33e9ec4b9da2563577db0499d880171f324bdf43f9e19d...,2020-01-01
1233283,2.965865e+10,96655,46379400,2014-06-04,NaN,NaN,33e9ec4b9da2563577db0499d880171f324bdf43f9e19d...,2020-01-01
1233963,2.830239e+10,96655,46379400,2004-08-23,NaN,NaN,33e9ec4b9da2563577db0499d880171f324bdf43f9e19d...,2020-01-01
1234741,1.813044e+10,96655,46379400,2012-02-01,NaN,NaN,33e9ec4b9da2563577db0499d880171f324bdf43f9e19d...,2020-01-01
...,...,...,...,...,...,...,...,...
1732750,1.274843e+10,96655,46379400,2014-07-14,NaN,NaN,33e9ec4b9da2563577db0499d880171f324bdf43f9e19d...,2020-01-01
1733509,2.741082e+10,96655,46379400,2018-01-19,NaN,NaN,33e9ec4b9da2563577db0499d880171f324bdf43f9e19d...,2020-01-01
1734227,2.739366e+10,96655,46379400,2018-02-01,NaN,NaN,33e9ec4b9da2563577db0499d880171f324bdf43f9e19d...,2020-01-01
1734365,2.188159e+10,96655,46379400,2007-02-12,NaN,NaN,33e9ec4b9da2563577db0499d880171f324bdf43f9e19d...,2020-01-01


In [ ]:
cpf_total[cpf_total.CPF=="44198631883"]

,CPF,nu_cpf
12128675,44198631883,4.419863e+10


In [ ]:
csup[csup.nu_cpf==44198631883]

,co_aluno,nu_cpf,sexo,cor_raca,cor_raca_ces,cor_raca_enem,conclusao_em,ano_conclusao_em,idade,ano_nascimento,...,prouni_2017,prouni_2018,prouni_2019,apoio_social_2013,apoio_social_2014,apoio_social_2015,apoio_social_2016,apoio_social_2017,apoio_social_2018,apoio_social_2019


In [ ]:
len(pd.unique(csup.nu_cpf))

2763108

In [ ]:
len(pd.unique(cpf_total.nu_cpf))

54999886